In [172]:
import pandas as pd

## LECTURA DE TODOS LOS CSV A UTILIZAR

In [173]:
results = pd.read_csv('Formula 1 Actualizad/results.csv')
qualifying = pd.read_csv('Formula 1 Actualizad/qualifying.csv')
races = pd.read_csv('Formula 1 Actualizad/races.csv')
constructors = pd.read_csv('Formula 1 Actualizad/constructors.csv')
drivers = pd.read_csv('Formula 1 Actualizad/drivers.csv')
lap_times = pd.read_csv('Formula 1 Actualizad/lap_times.csv')
circuits = pd.read_csv('Formula 1 Actualizad/circuits.csv')
driver_standings = pd.read_csv("Formula 1 Actualizad/driver_standings.csv")
constructor_standings = pd.read_csv("Formula 1 Actualizad/constructor_standings.csv")

#### MERGE 1 - dff1 
+ Race results : To obtain positions, resultId to identify every observation, points, rythm, fastest lap and status
+ qualifying results : to obtain qualifying positions and lap times

**On raceId, driverId and constructorId, the skeleton of most of our merges**

In [174]:
dff1 = pd.merge(results, qualifying, on=['raceId', 'driverId', 'constructorId'], how= 'outer')

In [175]:
dff1

,resultId,raceId,driverId,constructorId,number_x,grid,position_x,positionText,positionOrder,points,...,rank,fastestLapTime,fastestLapSpeed,statusId,qualifyId,number_y,position_y,q1,q2,q3
0,1.0,18,1,1,22,1.0,1,1,1.0,10.0,...,2,1:27.452,218.300,1.0,1.0,22.0,1.0,1:26.572,1:25.187,1:26.714
1,2.0,18,2,2,3,5.0,2,2,2.0,8.0,...,3,1:27.739,217.586,1.0,5.0,3.0,5.0,1:25.960,1:25.518,1:27.236
2,3.0,18,3,3,7,7.0,3,3,3.0,6.0,...,5,1:28.090,216.719,1.0,7.0,7.0,7.0,1:26.295,1:26.059,1:28.687
3,4.0,18,4,4,5,11.0,4,4,4.0,5.0,...,7,1:28.603,215.464,1.0,12.0,5.0,12.0,1:26.907,1:26.188,\N
4,5.0,18,5,1,23,3.0,5,5,5.0,4.0,...,1,1:27.418,218.385,1.0,3.0,23.0,3.0,1:25.664,1:25.452,1:27.079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25845,NaN,942,829,206,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6621.0,28.0,19.0,2:04.526,\N,\N
25846,NaN,943,834,206,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6640.0,53.0,18.0,1:24.136,\N,\N
25847,NaN,943,829,206,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6641.0,28.0,19.0,1:24.386,\N,\N
25848,NaN,944,834,206,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6660.0,53.0,18.0,1:16.151,\N,\N


#### MERGE 2 - dff2
+ dff1 (results + qualifying)
+ races : to obtain year and GP order (column 'round')

**Extra steps:** 
1. Drop every unnecesary column by now (dates and times of different events, driver number -because we already have the Id-, urls)
2. Drop every row from seasons before 2017

In [1]:
dff2 = pd.merge(dff1, races, on='raceId')
dff2.drop(['time_y', 'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time',
       'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date',
       'sprint_time', 'name', 'date', 'number_y', 'number_x', 'position_x'], axis=1, inplace=True)
dff2 = dff2[dff2['year']>=2017]

NameError: name 'pd' is not defined

#### MERGE 3 - dff3

+ dff2 (results + qualifying + races)
+ constructors: to associate the name of the team with every constructorId for a better visual understanding of the dataFrame

In [177]:
dff3 = pd.merge(dff2, constructors[['constructorId', 'name']], on = 'constructorId')

#### MERGE 4 - dff4

+ dff3 (results + qualifying + races + constructors)
+ drivers: to associate drivers surnames with every driversId for a better visual understanding of the dataFrame

In [178]:
dff4 = pd.merge(dff3, drivers[['driverId', 'surname']], on='driverId')

#### MERGE 5 - dff5

**Previous step:** groupby of every laptimes per driver per race, taking the average value of all of them

+ dff4 (results + qualifying + races + constructors)
+ average_laptimes

In [179]:
average_laptimes = lap_times.groupby(['raceId', 'driverId'])[['milliseconds']].mean()
average_laptimes.reset_index(level=['raceId', 'driverId'], inplace=True)
dff5 = pd.merge(dff4, average_laptimes, on= ['raceId', 'driverId'], how='left')

#### MERGE 6 - dff6
+ dff5 (results + qualifying + races + constructors + average lap times)
+ circuits: to associate circuit names with every circuitId for a better visual understanding of the dataFrame

In [180]:
dff6 = pd.merge(dff5, circuits[['circuitId', 'circuitRef']], on='circuitId')

#### MERGE 7 - dff7

+ dff6 (results + qualifying + races + constructors + average lap times + circuits)
+ driver_standings: to obtain points and number of victories of each driver

In [181]:
dff7 = pd.merge(dff6, driver_standings, on= ['raceId','driverId'])

#### MERGE 8 - dff8
+ dff7 (results + qualifying + races + constructors + average lap times + circuits + driver_standings)
+ constructor standings: to obtain points and number of victories of each team

In [182]:
dff8 = pd.merge(dff7, constructor_standings, on = ['raceId', 'constructorId'], suffixes=['_driver', '_constructor'])

## COLUMNAS NUEVAS
1. Abandono

In [183]:
dff8['DNF'] = dff8['positionText_x'][dff8['positionText_x']== 'R']
dff8['DNF'].replace('R', 1, inplace=True)
dff8['DNF'].fillna(0, inplace=True)

2. Puntos/Carrera de cada piloto en el campeonato, antes de la carrera a predecir

In [184]:
dff8['Pts Cmp'] = (dff8['points_y'] - dff8['points_x']) / (dff8['round']-1)

3. Puntos de cada equipo en el campeonato, antes de la carrera a predecir

In [185]:
target_mean = dff8.groupby(['raceId', 'constructorId'])['points_x'].sum().rename('Puntos ganados en esa carrera')

dff8 = dff8.join(target_mean, on=['raceId', 'constructorId'])

dff8['Pts Con'] = (dff8['points'] - dff8['Puntos ganados en esa carrera'])/ (dff8['round']-1)

dff8['Pts Con'].fillna(0, inplace= True)
dff8['Pts Cmp'].fillna(0, inplace=True)

In [186]:
dff8.sort_values(by='resultId', inplace=True)

4. Start from pitlane (bool)

We realize that starting from pitlane for is defined by our dataset as starting in position 0, which does not make any sense, so we consider it as the position 21, because it is much worse than being in the last position of the grid, and to create a boolean about it.

In [187]:
dff8['grid'].replace(0, 21, inplace=True)

In [188]:
dff8['Start From Pitlane'] = 0
dff8['Start From Pitlane'].where(dff8['grid'] != 21, 1, inplace=True)

5. Gained/Lost positions

In [189]:
dff8['Gained/Lost positions'] = dff8['grid'] - dff8['positionOrder']
dff8['Total Gained/Lost positions'] = dff8.groupby(['driverId', 'year'])['Gained/Lost positions'].cumsum()
dff8['TTotal Gained/Lost positions'] = dff8['Total Gained/Lost positions'] - dff8['Gained/Lost positions']

6. DNF INDEX (DNF PER DRIVER/TEAM * AVERAGE DNF PER CIRCUIT)

In [190]:
dnf_driver_mean = dff8.groupby(['year', 'driverId'])['DNF'].mean().rename('DNF/driver')

dff8 = dff8.join(dnf_driver_mean, on=['year', 'driverId'])

dnf_circuit_mean = dff8.groupby(['year', 'circuitId'])['DNF'].sum().rename('DNF/circuit')

dff8 = dff8.join(dnf_circuit_mean, on=['year', 'circuitId'])

dnf_constructor_mean = dff8.groupby(['year', 'constructorId'])['DNF'].mean().rename('DNF/constructor')

dff8 = dff8.join(dnf_constructor_mean, on=['year', 'constructorId'])

In [191]:
dff8['Index DNF/Driver'] = dff8['DNF/driver'] * dff8['DNF/circuit']
dff8['Index DNF/Constructor'] = dff8['DNF/constructor'] * dff8['DNF/circuit']

7. Average Lost Laps in the season

In [192]:
dff8['Lost Laps'] = 0
dff8['Lost Laps'].where(dff8['statusId'] != 11, 1, inplace=True)
dff8['Lost Laps'].where(dff8['statusId'] != 12, 2, inplace=True)
dff8['Lost Laps'].where(dff8['statusId'] != 13, 3, inplace=True)
dff8['Lost Laps'].where(dff8['statusId'] != 14, 4, inplace=True)
dff8['Lost Laps'].where(dff8['statusId'] != 15, 5, inplace=True)
dff8['Lost Laps'].where(dff8['statusId'] != 16, 6, inplace=True)

dff8['Sum of Lost Laps'] = dff8.groupby(['driverId', 'year'])['Lost Laps'].cumsum()
dff8['Sum of Lost Laps before the race'] = dff8['Sum of Lost Laps'] - dff8['Lost Laps']
dff8['Average Lost Laps'] = dff8['Sum of Lost Laps before the race'] / (dff8['round'] - 1)
dff8['Average Lost Laps'].fillna(0, inplace=True)

In [193]:
dff8['position_y'][dff8['position_y'].isna()] = 20

C:\Users\ramon\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


8. Average Speed per circuit (Source: F1) 
* No correlation found

In [194]:
dff8['circuitRef'].unique()
dff8['Circuit_speed'] = dff8['circuitRef'].map({'albert_park': 211, 'bahrain': 201, 'shanghai': 194, 
      'baku': 202, 'catalunya': 193, 'monaco': 153, 'villeneuve': 202, 'ricard': 219, 'red_bull_ring':225,
      'silverstone':225, 'hockenheimring': 207, 'hungaroring':186, 'spa':224, 'monza':241, 'marina_bay':164,
      'sochi':211, 'suzuka': 215, 'americas': 196, 'rodriguez': 190, 'interlagos': 206,
      'yas_marina': 192, 'mugello': 232, 'nurburgring': 196, 'portimao': 207, 'imola': 224,
      'istanbul':207, 'zandvoort': 205, 'losail': 223, 'jeddah':240, 'miami':209})

9. Position in the last races

In [195]:
import pandas as pd

def add_previous_races(df, n):
    """
    Add columns to the dataframe with the results of the last n races for each driver.
    
    Parameters:
    df (DataFrame): The input dataframe with driver results.
    n (int): The number of previous races to include in the output dataframe.
    
    Returns:
    None
    """
    # Group the dataframe by the driver column
    grouped = df.groupby('driverId')
    
    # Initialize a list to store the shifted values
    shifted_values = []
    
    # Shift the values within each group n times to get the results of the previous races
    for i in range(1, n + 1):
        shifted_values.append(grouped['positionOrder'].shift(i))
    
    # Concatenate the shifted values with the original dataframe
    new_cols = [f'Result_Race-{i}' for i in range(1, n + 1)]
    for i, col in enumerate(new_cols):
        df[col] = shifted_values[i]
    
    return None

# Call the function to add the results of the last 5 races as new columns in the existing dataframe
add_previous_races(dff8, 5)

# Print the resulting dataframe

In [196]:
dff8

,resultId,raceId,driverId,constructorId,grid,positionText_x,positionOrder,points_x,laps,time_x,...,Lost Laps,Sum of Lost Laps,Sum of Lost Laps before the race,Average Lost Laps,Circuit_speed,Result_Race-1,Result_Race-2,Result_Race-3,Result_Race-4,Result_Race-5
0,23379.0,969,20,6,2.0,1,1.0,25.0,57.0,1:24:11.672,...,0,0,0,0.000000,211.0,NaN,NaN,NaN,NaN,NaN
20,23380.0,969,1,131,1.0,2,2.0,18.0,57.0,+9.975,...,0,0,0,0.000000,211.0,NaN,NaN,NaN,NaN,NaN
21,23381.0,969,822,131,3.0,3,3.0,15.0,57.0,+11.250,...,0,0,0,0.000000,211.0,NaN,NaN,NaN,NaN,NaN
1,23382.0,969,8,6,4.0,4,4.0,12.0,57.0,+22.393,...,0,0,0,0.000000,211.0,NaN,NaN,NaN,NaN,NaN
32,23383.0,969,830,9,5.0,5,5.0,10.0,57.0,+28.827,...,0,0,0,0.000000,211.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,25841.0,1096,854,210,12.0,16,16.0,0.0,57.0,\N,...,1,10,9,0.428571,192.0,13.0,16.0,15.0,17.0,13.0
2036,25842.0,1096,825,210,16.0,17,17.0,0.0,57.0,\N,...,1,9,8,0.380952,192.0,19.0,17.0,9.0,14.0,12.0
1962,25843.0,1096,1,131,5.0,18,18.0,0.0,55.0,\N,...,0,1,1,0.047619,192.0,2.0,2.0,2.0,5.0,9.0
2005,25844.0,1096,849,3,20.0,19,19.0,0.0,55.0,\N,...,0,12,12,0.571429,192.0,16.0,18.0,17.0,9.0,19.0


In [197]:
dffinal = pd.DataFrame({'ResultId': dff8['resultId'], 'RaceId': dff8['raceId'], 'Year': dff8['year'], 'GP' : dff8['round'],
                'CircuitId': dff8['circuitId'], 'DriverId': dff8['driverId'], 'ConstructorId': dff8['constructorId'],
                'Constructor Name': dff8['name'], 'Driver Name':dff8['surname'], 'Circuit': dff8['circuitRef'],
                'Final position': dff8['positionOrder'], 'DNF': dff8['DNF'], 
                'Fastest Lap Rank': dff8['rank'], 'Grid position': dff8['grid'], 'Average pace': dff8['milliseconds_y'], 
                'Q1': dff8['q1'], 'Q2': dff8['q2'], 'Q3': dff8['q3'], 'Qualifying position': dff8['position_y'],
                'Victories': dff8['wins_driver'], 'Drivers points': dff8['Pts Cmp'],
                'Constructors points': dff8['Pts Con'], 'Constructors victories': dff8['wins_constructor'],
                'Start from Pitlane': dff8['Start From Pitlane'], 
                'Total gained/lost positions': dff8['TTotal Gained/Lost positions'], 
                'Index DNF/Driver': dff8['Index DNF/Driver'], 'Index DNF/Constructor': dff8['Index DNF/Constructor'],
                'Average Lost laps': dff8['Average Lost Laps'], 'Circuit Speed': dff8['Circuit_speed'],
                'Result_Race-1': dff8['Result_Race-1'], 'Result_Race-2': dff8['Result_Race-2'],
                'Result_Race-3': dff8['Result_Race-3'], 'Result_Race-4': dff8['Result_Race-4'],
                'Result_Race-5': dff8['Result_Race-5']})
dffinal.columns

Index(['ResultId', 'RaceId', 'Year', 'GP', 'CircuitId', 'DriverId',
       'ConstructorId', 'Constructor Name', 'Driver Name', 'Circuit',
       'Final position', 'DNF', 'Fastest Lap Rank', 'Grid position',
       'Average pace', 'Q1', 'Q2', 'Q3', 'Qualifying position', 'Victories',
       'Drivers points', 'Constructors points', 'Constructors victories',
       'Start from Pitlane', 'Total gained/lost positions', 'Index DNF/Driver',
       'Index DNF/Constructor', 'Average Lost laps', 'Circuit Speed',
       'Result_Race-1', 'Result_Race-2', 'Result_Race-3', 'Result_Race-4',
       'Result_Race-5'],
      dtype='object')

In [198]:
dffinal.to_csv('F1.csv', header=True, index=True)